In [ ]:
# general imports

import io
import sys
import subprocess 

import numpy as np
import pylab as pl
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as a3

import galois
import json

from pyvista import examples
from collections import namedtuple
from vtk.util import numpy_support
from mpl_toolkits.mplot3d import Axes3D

# make sure the user has at least Python 3.7, which we need for some capture_output shennanigans below
assert sys.version_info.major == 3, "This requires at least Python 3.7"
assert sys.version_info.minor >= 7, "This requires at least Python 3.7"

In [ ]:
# viz stuff

# Let's define some colors
RED = '#FF0000'
YELLOW = '#CCCC00'
GREEN = '#00CC00'
CYAN = '#00CCCC'
BLUE = '#0000CC'
MAGENTA = '#CC00CC'

DK_RED = '#AA0000'
ORANGE = '#FF8400'
DK_GREEN = '#004400'
NEON_GREEN = '#00FE94'
LT_BLUE = '#5555FF'
NEON_BLUE = '#7DF9FF'
LIGHT_COLOR = 'white'
BACKGROUND_COLOR = 'white'

L_color = '#CC0000'
W_color = '#660000'
C_color = '#009900'
V1_color = '#006600'
V2_color = '#2222FF'

W_color = RED
C_color = GREEN
V1_color = GREEN
V1_color = BLUE

W_V1_color = ORANGE
V1_V1_color = YELLOW
V1_V2_color = BLUE
V2_V2_color = GREEN

LINE_COLOR = 'black'
OTHER_LINE_COLOR = 'black'

vertex_size = 25
line_size = 3
other_line_size = 4


def visualize_graph(vertex_position, inc_matrix, vertex_value, L, W, C, V1, V2, lines, fname, camera_pos, camera_rot):
    light1 = pv.Light(position=(10, 10., -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 5400 K
                       intensity=1.5)
    light2 = pv.Light(position=(-10, 10.0, -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 2850 K
                       intensity=0.75)

    pl = pv.Plotter(lighting=None, window_size=(2000, 2000))
    #pl.add_light(light1)  # Lighting effect to be cast onto the object
    #pl.add_light(light2)  # Second Lighting effect to be cast onto the object
    pl.set_background(BACKGROUND_COLOR)  # Set the background 

    num_vertices = len(vertex_position)
    L_vertex_position=[]
    for i in L:
        L_vertex_position.append(vertex_position[i])
    W_vertex_position=[]
    for i in W:
        W_vertex_position.append(vertex_position[i])
    C_vertex_position=[]
    for i in C:
        C_vertex_position.append(vertex_position[i])
    V1_vertex_position=[]
    for i in V1:
        V1_vertex_position.append(vertex_position[i])
    V2_vertex_position=[]
    for i in V2:
        V2_vertex_position.append(vertex_position[i])
        
            
               
    pdata = pv.PolyData(vertex_position)    
#this adds point labels of vector values -- but placed on top of the points
#vertex_value must be passed in to visualize_graph
#     point_labels = []
#     for i in range(len(vertex_value)):
#         point_labels.append(vertex_value[i])        
#     pl.add_point_labels(pdata, point_labels, italic=False, bold=True, font_size=30,
#                         point_color='black', point_size=0, text_color='black', 
#                         shape=None, 
#                         render_points_as_spheres=True,
#                         always_visible=True, shadow=False)   
    
    pdata.lines = lines
    pl.add_mesh(pdata,
                color=LINE_COLOR, 
                #opacity=0.08,
                opacity=0.2,
                point_size=0,
                line_width = line_size,
                render_points_as_spheres=True
    )
    
    # this makes some lines whatever color you want
    counter = 0
    num_vertices = len(inc_matrix[0])
    other_vertex_pointer = []
    other_vertex_position = []
    other_lines = []
    # get L
    other_vertex_pointer.append(L[0])
    other_vertex_position.append(vertex_position[L[0]])
    counter += 1
    # get its nearest neighbor
    other_vertex_pointer.append(L[0]+1)
    other_vertex_position.append(vertex_position[L[0]+1])
    other_lines.append([2, 0, 1]) 
    counter += 1    
#     # get L neighbors (in C) with their lines
#     for i in range(num_vertices):
#         if inc_matrix[L[0]][i]==1 and L[0]!=i:
#             other_vertex_pointer.append(i)
#             other_vertex_position.append(vertex_position[i])
#             other_lines.append([2, 0, counter]) 
#             counter += 1
     # choose the last L neighbor and get its non-quadric neighbors with their lines
    num_other = len(other_vertex_pointer)
    num_chosen_neighbor = num_other-1
    for i in range(num_vertices):
        if inc_matrix[other_vertex_pointer[num_chosen_neighbor]][i]==1 and (i not in L) and (i not in W):
            other_vertex_pointer.append(i)
            other_vertex_position.append(vertex_position[i]) 
            other_lines.append([2, num_chosen_neighbor, counter]) 
            counter += 1
     # get the rest of the lines in the cluster
    num_other = len(other_vertex_pointer)
    for i in range(num_other):
        for j in range(num_other):
            if (inc_matrix[other_vertex_pointer[i]][other_vertex_pointer[j]]==1):
                other_lines.append([2, i, j]) 
    other_pdata = pv.PolyData(other_vertex_position)
    other_pdata.lines = other_lines
#     pl.add_mesh(other_pdata,
#                 color=LINE_COLOR,
#                 opacity=1.0,
#                 point_size=0,
#                 line_width = other_line_size,
#                 render_points_as_spheres=True
#     )

    
    
    L_cloud = pv.PolyData(L_vertex_position)
    pl.add_mesh(L_cloud, 
                color=L_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    W_cloud = pv.PolyData(W_vertex_position)
    pl.add_mesh(W_cloud, 
                color=W_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    C_cloud = pv.PolyData(C_vertex_position)
    pl.add_mesh(C_cloud, 
                color=C_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    V1_cloud = pv.PolyData(V1_vertex_position)
    pl.add_mesh(V1_cloud, 
                color=V1_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    V2_cloud = pv.PolyData(V2_vertex_position)
    pl.add_mesh(V2_cloud, 
                color=V2_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    # Views: xy, xz, yz, yx, zx, zy, iso 
    pl.camera_position = camera_pos
    pl.camera.roll = camera_rot[0]
    pl.camera.azimuth = camera_rot[1]
    pl.camera.elevation = camera_rot[2]
    pl.camera.zoom(1)
   
    # Save the plot as a screenshot.  We can change the output resolution if desired
    pl.screenshot(fname, window_size=[2000,2000])
    #pl.add_point_labels(vertices, range(num_vertices), font_size=100)
    #pl.show_axes()
    pl.show()
    


In [ ]:
# finite field stuff, {F_q}^3

# addition and multiplication tables for F_q
# for now just hard-coding these for small fields, later can generalize
# python does have an extension for Galois fields, see https://pypi.org/project/galois/

def make_GFq_arith_table(q):
    GFq_add = []
    GFq_mul = []
    GF = galois.GF(q)
    for i in range(q):
        x_vec = []
        y_vec = []
        for j in range(q):
            x_vec.append(i)
            y_vec.append(j)
        x = GF(x_vec)
        y = GF(y_vec)
        GFq_add.append(json.loads(str(x+y)))
        GFq_mul.append(json.loads(str(x*y)))
    return GFq_add, GFq_mul
    
def dot_product_3d(v,w,q):
    s=[]
    for i in range(3):
        s.append(GFq_mul[v[i]][w[i]])
    d = GFq_add[GFq_add[s[0]][s[1]]][s[2]]
    return d


In [ ]:
# graph stuff, generating L, W, C, V1 and V2
# also permuting the canonical ordering to give a good initial layout that illustrates points to make

def generate_vertices(q, num_vertices):
# this generates the q^2+q+1 vertices of ER_q
# vertex_value gives the left-normalized vectors of ER_q, in lexicographic order
# vertex_position gives the initial positions of q^2+q+1 points equidistant on the unit circle
# you can permute vertex_value to give another ordering of the left-normalized vectors of ER_q
# you can change vertex_position to place these elsewhere in the space, for example into 3-space

    vertex_value = []
    vertex_position = []
    # vertices locations
    for i in range(num_vertices):
        angle_inc = 2*np.pi/num_vertices    
        vertex_position.append([np.sin(i*angle_inc), np.cos(i*angle_inc), 0])
    # vertices values as vectors in {F_q}^3
    for i in range(int(q**3)):
        new_point = []
        new_point.insert(0, int(i%q))
        new_point.insert(0, int(((i-new_point[0])/q)%q))
        new_point.insert(0, int(((i-new_point[0]*q-new_point[1])/q**2)%q))
        if (new_point[0]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==0 and new_point[2]==1):
            vertex_value.append(new_point)            
    return vertex_position, vertex_value, angle_inc

def permute_vertices_values(vertex_value, q):
# this function generates an initial layout that is attractive and explanatory for our purposes.
# the lexicographic layout around the circle seems not to expose the structures of interest.
# vv_permute_matrix[i][j] means the permutation sends the jth element to the ith position
# returns the permutation matrix vv_permute_matrix and its inverse vv_permute_inverse
    
    num_vertices = len(vertex_value)    
    # the permutation matrix for the initial layout, initialized to 0
    vv_permute = []
    vv_permute_inverse = []
    for i in range(len(vertex_value)):
        row=[]
        row_inv=[]
        for j in range(len(vertex_value)):
            row.append(0)
            row_inv.append(0)
        vv_permute.append(row)
        vv_permute_inverse.append(row_inv)

    # this generates W and picks an arbitrary element of W to be the "starter" quadric in L
    L,W = generate_W(inc_matrix)
    # this generates V1 and grabs the center set based on L
    C,V1 = generate_V1(L, W, inc_matrix)
    # V2, the rest of the vertices
    V2 = generate_V2(L, W, C, V1, num_vertices)
    
    # a bunch of cycles    
    # the initial layout is mapped onto a circle, with q^2+q+1 equidistant points on it
    
    # move the quadric in L to 1+(q-1)/2 points to the left of the top of the circle
    vv_permute[num_vertices-1-int((q-1)/2)][L[0]] = 1
    
    # move the q center elements C to the q points covering the top of the circle
    end_C = int((q-1)/2)
    for i in range(len(C)):
        #print(C[i],(num_vertices-int((q-1)/2)+i)%num_vertices)
        vv_permute[(i-end_C)%num_vertices][C[i]] = 1
    current_pos = end_C+1
    
    # move the rest of the elements of V1 to points corresponding to their centers
    # in other words, look at the reflection of a center about the x axis
    # then move its points from V1 to flank that point
    # in ith cluster (ith C element):
    for i in range(len(C)): 
        current_pos+=1
        V1_temp=[]
        V2_temp=[]
        # get all the elts of V1 and V2 into V1_temp and V2_temp
        for j in range(len(V1)):
            if inc_matrix[V1[j]][C[len(C)-1-i]]==1:
                V1_temp.append(V1[j])
            if inc_matrix[V2[j]][C[len(C)-1-i]]==1:
                V2_temp.append(V2[j])
        # make triangles
        #current_pos = end_C+1       
        if (q%4 != 1):
            for j in range(int((q-1)/2)):
                # take the jth V1_temp and match it up with the V2 that is orthogonal
                # here's the jth V1
                vv_permute[current_pos][V1_temp[j]] = 1
                current_pos += 1
                # take the orthogonal V2
                for k in range(len(V2_temp)):
                    if inc_matrix[V1_temp[j]][V2_temp[k]]==1:
                        vv_permute[current_pos][V2_temp[k]] = 1
                        current_pos += 1
                        break
        else: # q%4 == 1
            # will have (q-1)/4 pairs of V1 only tris, and (q-1)/4 pairs of V2 only tris
            for j in range(int((q-1)/4)):
                # all the tris are within V1 or within V2
                # pick the first V1 and V2 (any one not picked will do)
                vv_permute[current_pos][V1_temp[0]] = 1
                current_pos += 1
                vv_permute[current_pos][V2_temp[0]] = 1
                current_pos += 1
                #pick twins of V1_temp[0]
                for k in range(1,len(V1_temp)):
                    if inc_matrix[V1_temp[0]][V1_temp[k]]==1:
                        vv_permute[current_pos][V1_temp[k]] = 1
                        current_pos += 1
                        V1_temp.pop(k)
                        V1_temp.pop(0)
                        break
                #pick twins of V2_temp[0]
                for k in range(1,len(V2_temp)):
                    if inc_matrix[V2_temp[0]][V2_temp[k]]==1:
                        vv_permute[current_pos][V2_temp[k]] = 1
                        current_pos += 1
                        V2_temp.pop(k)
                        V2_temp.pop(0)
                        break                
        
    # now place the remaining q elements of W
    # these are interspersed between each "rack" C_i of V1 and V2 points
    C_temp=[]
    for i in range(len(C)):
        C_temp.append(C[i])
    mid_C = int((len(C)-1)/2)
    C_temp.pop(mid_C)
    for i in range(len(W)):
        if inc_matrix[W[i]][C[mid_C]]==1:
            vv_permute[(end_C+1)%num_vertices][W[i]] = 1
    for j in range(len(C_temp)):
        W_temp=[]
        for i in range(len(W)):
            if inc_matrix[W[i]][C_temp[len(C_temp)-1-j]]==1:
                W_temp.append(W[i])                                
        for i in range(len(W_temp)):
            vv_permute[(end_C+1+(j+1)*q)%num_vertices][W_temp[i]] = 1        
    for i in range(len(vertex_value)):
        for j in range(len(vertex_value)):
            vv_permute_inverse[j][i]=vv_permute[i][j]         
    return (vv_permute, vv_permute_inverse)

def incidence_matrix(vertices_values):
    inc_matrix = []
    num_vertices = len(vertices_values)
    for i in range(num_vertices):
        row=[]
        for j in range(num_vertices):
            row.append(0)
        inc_matrix.append(row)
    for i in range(num_vertices):
        for j in range(i,num_vertices):
            d = dot_product_3d(vertices_values[i], vertices_values[j],q)
            if d==0:
                inc_matrix[i][j]=1
                inc_matrix[j][i]=1
    return inc_matrix

def generate_W(inc_matrix):
    W=[]
    L=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if inc_matrix[i][i]==1:
            W.append(i)
    # grab the last element of W to be the chosen quadric
    L.append(W[len(W)-1])
    W.pop(len(W)-1)    
    return (L,W)

def generate_V1(L, W, inc_matrix):
    C=[]
    V1=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and inc_matrix[i][L[0]]==1:
            C.append(i)     
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C):
            for j in W:
                if inc_matrix[i][j]==1:
                    V1.append(i)
                    break
    return (C,V1)

def generate_V2(L, W, C, V1, num_vertices):
    V2=[]
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C) and (i not in V1):
            V2.append(i)
    return V2

def reset_positions(vertex_position):
    v_pos_temp = []
    for i in range(len(vertex_position)):
        v_pos_temp.append(vertex_position[i])
    return v_pos_temp


In [1]:
# these are the prime powers q to be run
#prime_power_list = [3,5,7,9]
#prime_power_list = [3,5,7,9,11,13,17,19,23]
prime_power_list = [9]

for q in prime_power_list:
    GFq_add,GFq_mul=make_GFq_arith_table(q)
#     for i in range(q):
#         print(GFq_add[i])
#     print()
#     for i in range(q):
#         print(GFq_mul[i])
#     print()
    
    num_vertices = q**2+q+1

    # here are the vertices and their positions on the unit circle
    vertex_position, vertex_value, base_angle = generate_vertices(q, num_vertices)
    # here are the edges
    inc_matrix = incidence_matrix(vertex_value)
    
    # get the permutation matrix for the vector ordering for a nice initial layout
    vv_permute_matrix, vv_permute_matrix_inverse = permute_vertices_values(vertex_value, q)
        
    # permute the vertex values
    vertex_value_permuted = []
    for i in range(len(vertex_value)):
        vertex_value_permuted.append([0,0,0])
    for i in range(len(vertex_value)):
        for j in range(len(vertex_value)):
            if (vv_permute_matrix[j][i]==1):
                new_index = j
        vertex_value_permuted[new_index] = vertex_value[i]
    
    # make an incidence matrix (edges) for the permuted vertices
    inc_matrix_permuted = incidence_matrix(vertex_value_permuted)
    lines = []
    for i in range(num_vertices):
        for j in range(i+1,num_vertices):
            if inc_matrix_permuted[i][j]==1:
                lines.append([2, i, j])

    # L is the arbitary quadric that starts things off
    # W is the rest of the quadrics
    # C is the set of centers: the vectors that are adjacent to the element in L
    # V1 is the set of all other vectors adjacent to some quadric
    # V2 is the rest of the vectors, not adjacent to any quadric
    L_values=[]
    W_values=[]
    C_values=[]
    V1_values=[]
    V2_values=[]
    L,W = generate_W(inc_matrix_permuted)
    for i in range(len(L)):
        L_values.append(vertex_value_permuted[L[i]])
    for i in range(len(W)):
        W_values.append(vertex_value_permuted[W[i]])
    C,V1 = generate_V1(L, W, inc_matrix_permuted)
    for i in range(len(C)):
        C_values.append(vertex_value_permuted[C[i]])
    for i in range(len(V1)):
        V1_values.append(vertex_value_permuted[V1[i]])
    V2 = generate_V2(L, W, C, V1, num_vertices)
    for i in range(len(V2)):
        V2_values.append(vertex_value_permuted[V2[i]])
       
    # *_values gives the $F_q$ vector values of the elements
    # *_positions gives their initial positions on the circle graphs
        
    print("GF (",q,")")
#     print("L_values = ", *L_values)
#     print("W_values = ", *W_values)
#     print("C_values = ", *C_values)
#     print("V1_values = ", *V1_values)
#     print("V2_values = ", *V2_values)
#     print("L pointers = ", *L)
#     print("W pointers = ", *W)
#     print("C pointers = ", *C)
#     print("V1 pointers = ", *V1)
#     print("V2 pointers = ", *V2)

    directory = "./ER_graphs/"
    fn_prefix = directory + "ER" + str(q)
    filename = fn_prefix + "_0_orig"
    # camera_pos views: xy, xz, yz, yx, zx, zy, iso 
    # We can also define a camera_pos = (x,y,z) tuple containing angle on each axis
    camera_pos = 'xy'
    # We can also define a camera rotation = (roll, azimuth, elevation) tuple containing angle in degrees about each axis
    camera_rot=[0,0,0]
    
# show the initial layout
# viz 0
    visualize_graph(vertex_position, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)
    
# vertex_position is a bunch of spatial positions and vertex_value_permuted is a bunch of vector values
# vertex_position[i] is the position in space of vertex_value_permuted[i]
# so change vertex_position[i] to move vertex_value_permuted[i] in the visualized graph
# this is the experimentation part for an attractive and illustrative layout
    
    v_pos_temp = reset_positions(vertex_position)
# move the arbitrary quadric element in L outside and above the circle
    height_L = 1.5
    v_pos_temp[L[0]] = [0.0,height_L,0.0]                 
# viz 1
    visualize_graph(v_pos_temp, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)

    
    # also move V1 and V2 to the same height
    height_V = 0.0
    for i in range(len(V1)):
        v_pos_temp[V1[i]][1] = height_V
    for i in range(len(V2)):
        v_pos_temp[V2[i]][1] = height_V
# viz 1.5
    visualize_graph(v_pos_temp, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)
    
    
# 0th W element moved to the bottom of the circle -- to move some distracting edges
    height_W = -1
    v_pos_temp[W[0]] = [0.0,height_W,0.0] 
    filename = fn_prefix + "_1_base_W"
    # all other W elements moved outside and below the circle
    height = -1.0
    for i in range(len(W)):
        v_pos_temp[W[i]][1] = height
#     vertex_position[W[0]] = [0.0,-1.75,0.0] 
#     half_W = int((len(W)-1)/2)
#     extreme = 3.0
#     W_space = 1.0
#     for i in range(1,half_W+1):
#         vertex_position[W[i]][0] = float(i)
#         vertex_position[W[i]][1] = -1.75
#     for i in range(half_W+1,len(W)):
#         vertex_position[W[i]][0] = float(half_W-i)
#         vertex_position[W[i]][1] = -1.75
    filename = fn_prefix + "_2_all_W"
# viz 2
    visualize_graph(v_pos_temp, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)
        
    # Move the V2 elements outside and to the bottom
    vertex_position_orig = vertex_position.copy()
    for i, vertex in enumerate(V2):
        vertex_position[V2[i]][1] -= 2.0
    filename = fn_prefix + "_3_V2_bottom"
# viz 3
    visualize_graph(vertex_position, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)

    # Move the V2 elements back then inside 
    for i, vertex in enumerate(V2):
        vertex_position[V2[i]][1] += 2.0
        for j in range(3):
            vertex_position[V2[i]][j] /= 2.5
    filename = fn_prefix + "_4_V2_inside"
# viz 4
    visualize_graph(vertex_position, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)

    # Move the V2 elements back to their original positions then out toward the viewer 
    for i, vertex in enumerate(V2):
        for j in range(3):
            vertex_position[V2[i]][j] *= 2.5
        vertex_position[V2[i]][2] += 1.0
        
    # Move the W elements away from the viewer but in line with the other two circles -- a 3D "tin can"

    # First move the 0th W element to the middle of the list -- to move some distracting edges
    temp_W0 = W[0]
    W0_new_index = int((q-1)/2)
    for i in range(W0_new_index):
        W[i] = W[i+1]
    W[W0_new_index] = temp_W0
    
    if (q != 1%4):
        offset = num_vertices-(q-1)/2-1-(q-1)/2
    else:
        offset = num_vertices-(q-1)/2 -1 -(q-1)/2-0.5
        #offset = q+0.5
    starting_angle = offset*base_angle
    # move the W away from the viewer and into the center
    num_W = len(W)
    radius_W = 0.2
    angle_W = 2*np.pi/(num_W) 
    vertex_position[L[0]]=[radius_W*np.sin(starting_angle),radius_W*np.cos(starting_angle), -1.0]
#    print("vertex_position[L[0]] is ", vertex_position[L[0]])
    for i in range(num_W):
        vertex_position[W[i]] = [radius_W*np.sin(i*angle_W+starting_angle), radius_W*np.cos(i*angle_W+starting_angle), -1.0]       

    # Move the C elements into the center
    offset = offset
    starting_angle = offset*base_angle
    starting_angle = 3*np.pi/4
    radius_C= 1.5*radius_W
    num_C = len(C)
    angle_C = 2*np.pi/(num_C) 
    vertex_position[C[0]]=[radius_C*np.sin(starting_angle),radius_C*np.cos(starting_angle), 0.0]
#    print("vertex_position[C[0]] is ", vertex_position[C[0]])
    for i in range(1,num_C):
        vertex_position[C[i]] = [radius_C*np.sin(i*angle_C+starting_angle), radius_C*np.cos(i*angle_C+starting_angle), 0.0]
#        vertex_position[C[i]] = vertex_position[C[0]]
#        vertex_position[C[i]] = [radius_C*np.sin((i+offset+3)*angle_C), radius_C*np.cos((i+offset+3)*angle_C), 0.0]

    filename = fn_prefix + "_5_3D"    
    # to get the interesting POV on this 3D graph, we have to play with the camera position
    camera_pos = 'zx'
    camera_rot = [0,0,23]
# viz 5
    visualize_graph(vertex_position, inc_matrix_permuted, vertex_value_permuted, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)



NameError: name 'make_GFq_arith_table' is not defined

In [ ]:
# This cell gives the lexicographic layout for the vector vertices.
# This layout doesn't illuminate the structures we want to discuss, so these are just for reference.

prime_power_list = [3,5,7,9]

for q in prime_power_list:
    num_vertices = q**2+q+1

    vertices, vertices_values = generate_vertices(q, num_vertices)
    inc_matrix = incidence_matrix(vertices_values)
    lines = []
    for i in range(num_vertices):
        for j in range(i+1,num_vertices):
            if inc_matrix[i][j]==1:
                lines.append([2, i, j])

    print("GF (",q,")")
    L,W = generate_W(inc_matrix)
    print("L = ", L)
    print("W = ", W)
    C,V1 = generate_V1(L, W, inc_matrix)
    print("V1 = ", V1)
    V2 = generate_V2(L, W, C, V1, num_vertices)
    print("V2 = ", V2)
    directory = "data/"
    fn_prefix = directory + "ER" + str(q)
    fname = fn_prefix + "_naive"
    # camera_pos views: xy, xz, yz, yx, zx, zy, iso 
    # We can also define a camera_pos = (x,y,z) tuple containing angle on each axis
    camera_pos = 'xy'
    # We can also define a camera rotation = (roll, azimuth, elevation) tuple containing angle in degrees about each axis
    camera_rot=[0,0,0]
    #visualize_graph(vertices, inc_matrix, vertices_values, L, W, C, V1, V2, lines, filename, camera_pos, camera_rot)
    

In [ ]:
add_Fq=[]
mul_Fq=[]

for i in range(100):
    add_Fq.append([])
    mul_Fq.append([])

add_Fq[3] = [[0,1,2],
             [1,2,0],
             [2,0,1]]


mul_Fq[3] =  [[0,0,0],
              [0,1,2],
              [0,2,1]]


add_Fq[5] = [[0,1,2,3,4],
             [1,2,3,4,0],
             [2,3,4,0,1],
             [3,4,0,1,2],
             [4,0,1,2,3]]

mul_Fq[5] =  [[0,0,0,0,0],
              [0,1,2,3,4],
              [0,2,4,1,3],
              [0,3,1,4,2],
              [0,4,3,2,1]]

add_Fq[7] = [[0,1,2,3,4,5,6],
             [1,2,3,4,5,6,0],
             [2,3,4,5,6,0,1],
             [3,4,5,6,0,1,2],
             [4,5,6,0,1,2,3],
             [5,6,0,1,2,3,4],
             [6,0,1,2,3,4,5]]

mul_Fq[7] = [[0,0,0,0,0,0,0],
             [0,1,2,3,4,5,6],
             [0,2,4,6,1,3,5],
             [0,3,6,2,5,1,4],
             [0,4,1,5,2,6,3],
             [0,5,3,1,6,4,2],
             [0,6,5,4,3,2,1]]

add_Fq[9] = [[0,1,2,3,4,5,6,7,8],
             [1,2,0,4,5,3,7,8,6],
             [2,0,1,5,3,4,8,6,7],
             [3,4,5,6,7,8,0,1,2],
             [4,5,3,7,8,6,1,2,0],
             [5,3,4,8,6,7,2,0,1],
             [6,7,8,0,1,2,3,4,5],
             [7,8,6,1,2,0,4,5,3],
             [8,6,7,2,0,1,5,3,4]]

mul_Fq[9] = [[0,0,0,0,0,0,0,0,0],
             [0,1,2,3,4,5,6,7,8],
             [0,2,1,6,8,7,3,5,4],
             [0,3,6,2,5,8,1,4,7],
             [0,4,8,5,6,1,7,2,3],
             [0,5,7,8,1,3,4,6,2],
             [0,6,3,1,7,4,2,8,5],
             [0,7,5,4,2,6,8,3,1],
             [0,8,4,7,3,2,5,1,6]]